# Deployment

Goal: Fill out this notebook to import your trained model and build a Gradio interface.

After mocking up Gradio interface: Deploy your model to HuggingFace Spaces

In [ ]:
!pip install gradio

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import * #used for fastai
from IPython import display #used to display media in notebook
import matplotlib.pyplot as plt #used to plot in notebook

from fastai.data.all import *
from fastai.data.external import *
from fastai.vision.all import *

import librosa;
import librosa.display;

import PIL as Pillow;
import gradio as gr;
import soundfile as sf;
import tempfile;

#### Load the learner from `.pkl` file. 

This will complain that you need some functions that aren't available in the namespace. Copy the necessary ones from the previous notebook.

In [ ]:
# TODO: put the functions that are missing from the namespace here
# ....

We have to define a prediction function for our model:

In [ ]:
learn = load_learner('model.pkl')

In [ ]:
labels = learn.dls.vocab

def predict(audio):
    # grab data from Gradio upload
    sample_rate, data = audio 
    
    # recall that our dataset loads paths to audio files first, not the files themselves
    # lets make a temporary (in-memory) file
    temp_file = tempfile.NamedTemporaryFile(suffix='.aiff')
    
    # use soundfile library to write to temp file
    sf.write(temp_file.name, data, sample_rate)
    
    # get our prediction results
    pred,pred_idx,probs = learn.predict(Path(temp_file.name))
    
    # close tempfile
    temp_file.close()
    
    # return prediction results
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [ ]:
title = "North Atlantic Right Whale Classifier"
description = "A NARW up-call classifier trained on the The Marinexplore and Cornell University Whale Detection Challenge dataset (Kaggle) with fastai."
article="<p style='text-align: center'><a href='https://www.kaggle.com/competitions/whale-detection-challenge' target='_blank'>Dataset</a></p>"
enable_queue=True

gr.Interface(fn=predict, inputs=gr.Audio(type="numpy"), outputs=gr.outputs.Label(num_top_classes=2),title=title,description=description,article=article,allow_flagging="never").launch(share=True)

#### Your Job

Deploy to HuggingFace Spaces after getting this Gradio notebook working